In [4]:
import argparse
import requests
import numpy as np
import pandas as pd
import hail as hl
from hail.expr.functions import _sort_by
from hail.utils import hadoop_open, new_temp_file
import pyranges as pr
from ukbb_pan_ancestry import (
    get_distance_clumping_results_path,
    get_known_ukbb_loci_path,
    get_meta_analysis_results_path,
    get_munged_otg_v2d_path,
    get_munged_round2_path,
    get_pheno_manifest_path,
    get_ukb_pheno_efo_mapping_path,
    load_final_sumstats_mt,
    otg_release,
)

Loading BokehJS ...

In [5]:
def _to_pandas(ht):
            df = ht.to_pandas()
            # a temporary fix: cf. https://github.com/biocore-ntnu/pyranges/pull/264
            df = df.astype({k: "object" if v == "string[python]" else str(v).lower() for k, v in df.dtypes.items()})
            return df

In [7]:
ht_v2d = hl.read_table("/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/munged_otg_v2d_no_exclusion.ht")
ht_v2d.show(3)

,,,,
locus,trait_efo,study_id,trait_efo_term,trait_efo_category
locus<GRCh37>,str,set<str>,str,str
1:768253,"""EFO_0006312""","{""GCST90026371""}","""mitochondrial DNA measurement""","""Other measurement"""
1:839538,"""EFO_0006865""","{""NEALE2_20002_1202""}","""urgency urinary incontinence""","""Other trait"""
1:858051,"""EFO_0004617""","{""GCST90025945""}","""cystatin c measurement""","""Other measurement"""


In [17]:
# otg lead variants
ht_v2d = hl.read_table("/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/munged_otg_v2d_no_exclusion.ht")
ht_v2d = ht_v2d.annotate(
    study_id=hl.delimit(ht_v2d.study_id),
    Chromosome=ht_v2d.locus.contig,
    Start=ht_v2d.locus.position,
    End=ht_v2d.locus.position,
)
ht_v2d.show(3)

,,,,,,,
locus,trait_efo,study_id,trait_efo_term,trait_efo_category,Chromosome,Start,End
locus<GRCh37>,str,str,str,str,str,int32,int32
1:768253,"""EFO_0006312""","""GCST90026371""","""mitochondrial DNA measurement""","""Other measurement""","""1""",768253,768253
1:839538,"""EFO_0006865""","""NEALE2_20002_1202""","""urgency urinary incontinence""","""Other trait""","""1""",839538,839538
1:858051,"""EFO_0004617""","""GCST90025945""","""cystatin c measurement""","""Other measurement""","""1""",858051,858051


In [15]:
df_v2d = _to_pandas(ht_v2d)
print(df_v2d.head())

      locus    trait_efo           study_id                    trait_efo_term  \
0  1:768253  EFO_0006312       GCST90026371     mitochondrial DNA measurement   
1  1:839538  EFO_0006865  NEALE2_20002_1202      urgency urinary incontinence   
2  1:858051  EFO_0004617       GCST90025945            cystatin c measurement   
3  1:891059  EFO_0004533       GCST90019494  alkaline phosphatase measurement   
4  1:894573  EFO_0002618         GCST005434              pancreatic carcinoma   

         trait_efo_category Chromosome   Start     End  
0         Other measurement          1  768253  768253  
1               Other trait          1  839538  839538  
2         Other measurement          1  858051  858051  
3  Liver enzyme measurement          1  891059  891059  
4                    Cancer          1  894573  894573  


In [20]:
#========== EAS biobanks as ref ===========#
ht_v2d = hl.import_table("/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/GWAS_Catalog/KoGES_otg_form.txt", delimiter='\t', impute=True)

# Strip the "Chr" prefix from the locus string (e.g., "Chr2:27730940" -> "2:27730940")
ht_v2d = ht_v2d.annotate(locus_stripped = hl.str(ht_v2d.locus).replace("^Chr", ""))

# Parse the modified locus field to a proper Hail locus object
ht_v2d = ht_v2d.annotate(locus = hl.parse_locus(ht_v2d.locus_stripped, reference_genome='GRCh37'))

# Now you can access the contig and position attributes
ht_v2d = ht_v2d.annotate(
    study_id = ht_v2d.study_id,
    Chromosome = ht_v2d.locus.contig,
    Start = ht_v2d.locus.position,
    End = ht_v2d.locus.position
)
df_v2d = _to_pandas(ht_v2d)
print(df_v2d.head())

         locus    trait_efo      study_id trait_efo_term trait_efo_category  \
0  15:89804417  EFO_0004339  GCST90255478    body height        body height   
1  15:90141591  EFO_0004339  GCST90255478    body height        body height   
2  15:90619081  EFO_0004339  GCST90255478    body height        body height   
3  15:99192483  EFO_0004339  GCST90255478    body height        body height   
4  15:99240993  EFO_0004339  GCST90255478    body height        body height   

  locus_stripped Chromosome     Start       End  
0    15:89804417         15  89804417  89804417  
1    15:90141591         15  90141591  90141591  
2    15:90619081         15  90619081  90619081  
3    15:99192483         15  99192483  99192483  
4    15:99240993         15  99240993  99240993  


In [6]:
#========== GWAS_Catalog as ref ===========#
trait_list = ["HEIGHT"]
study_list = ["Yengo"]
trait = trait_list[0]; study = study_list[0]
    
ht_v2d = hl.import_table(f"/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/GWAS_Catalog/{study}_{trait}_otg_form.txt", delimiter='\t', impute=True)
ht_v2d.show(3)

Initializing Hail with default parameters...


24/09/24 23:02:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.4
SparkUI available at http://holy7c24504.rc.fas.harvard.edu:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.130-bea04d9c79b5
LOGGING: writing to /n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/hail-20240924-2302-0.2.130-bea04d9c79b5.log
2024-09-24 23:03:22.827 Hail: INFO: Reading table to impute column types 0) / 1]
2024-09-24 23:03:24.276 Hail: INFO: Finished type imputation
  Loading field 'locus' as type str (imputed)
  Loading field 'trait_efo' as type str (imputed)
  Loading field 'study_id' as type str (imputed)
  Loading field 'trait_efo_term' as type str (imputed)
  Loading field 'trait_efo_category' as type str (imputed)


,,,,
locus,trait_efo,study_id,trait_efo_term,trait_efo_category
str,str,str,str,str
"""11:18634724""","""EFO_0004339""","""GCST90245845""","""body height""","""Body measurement"""
"""11:26821745""","""EFO_0004339""","""GCST90245845""","""body height""","""Body measurement"""
"""11:47447887""","""EFO_0004339""","""GCST90245845""","""body height""","""Body measurement"""


2024-09-24 23:25:34.650 Hail: INFO: Reading table to impute column types
2024-09-24 23:25:35.181 Hail: INFO: Finished type imputation
  Loading field 'locus' as type str (imputed)
  Loading field 'trait_efo' as type str (imputed)
  Loading field 'study_id' as type str (imputed)
  Loading field 'trait_efo_term' as type str (imputed)
  Loading field 'trait_efo_category' as type str (imputed)
2024-09-24 23:25:38.305 Hail: INFO: Reading table to impute column types
2024-09-24 23:25:49.268 Hail: INFO: Finished type imputation
  Loading field 'idx' as type int32 (imputed)
  Loading field 'lead_locus' as type str (imputed)
  Loading field 'region' as type str (imputed)
  Loading field 'trait_efo' as type str (imputed)
  Loading field 'trait_efo_category' as type str (imputed)
  Loading field 'locus' as type str (imputed)
  Loading field 'trait_efo_otg' as type str (imputed)
  Loading field 'study_id' as type str (imputed)
  Loading field 'trait_efo_term' as type str (imputed)
  Loading field 

In [10]:

# Strip the "Chr" prefix from the locus string (e.g., "Chr2:27730940" -> "2:27730940")
ht_v2d = ht_v2d.annotate(locus_stripped = hl.str(ht_v2d.locus).replace("^Chr", ""))

# Parse the modified locus field to a proper Hail locus object
ht_v2d = ht_v2d.annotate(locus = hl.parse_locus(ht_v2d.locus_stripped, reference_genome='GRCh37'))

# Now you can access the contig and position attributes
ht_v2d = ht_v2d.annotate(
    study_id = ht_v2d.study_id,
    Chromosome = ht_v2d.locus.contig,
    Start = ht_v2d.locus.position,
    End = ht_v2d.locus.position
)
df_v2d = _to_pandas(ht_v2d)
print(df_v2d.head())

         locus    trait_efo      study_id trait_efo_term trait_efo_category  \
0  11:18656271  EFO_0004339  GCST90245845    body height   Body measurement   
1  11:26843292  EFO_0004339  GCST90245845    body height   Body measurement   
2  11:47469439  EFO_0004339  GCST90245845    body height   Body measurement   
3  11:57753790  EFO_0004339  GCST90245845    body height   Body measurement   
4  11:61641542  EFO_0004339  GCST90245845    body height   Body measurement   

  locus_stripped Chromosome     Start       End  
0    11:18656271         11  18656271  18656271  
1    11:26843292         11  26843292  26843292  
2    11:47469439         11  47469439  47469439  
3    11:57753790         11  57753790  57753790  
4    11:61641542         11  61641542  61641542  


In [11]:
def annotate_known_loci_pmid(trait,study):

    #========== EAS biobanks as ref ===========#
    #ht_v2d = hl.import_table("/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/GWAS_Catalog/Yengo_HEIGHT_otg_form.txt", delimiter='\t', impute=True)
    ht_v2d = hl.import_table(f"/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/GWAS_Catalog/{study}_{trait}_otg_form.txt", delimiter='\t', impute=True)

    # Strip the "Chr" prefix from the locus string (e.g., "Chr2:27730940" -> "2:27730940")
    #ht_v2d = ht_v2d.annotate(locus_stripped=hl.str(ht_v2d.locus).replace("^Chr", ""))
    
    # Replace '23' with 'X' before parsing the locus
    ht_v2d = ht_v2d.annotate(
        locus=hl.if_else(ht_v2d.locus.startswith("23:"), 
                                  hl.str(ht_v2d.locus).replace("23:", "X:"), 
                                  ht_v2d.locus)
    )
    
    # Parse the modified locus field to a proper Hail locus object
    ht_v2d = ht_v2d.annotate(locus=hl.parse_locus(ht_v2d.locus, reference_genome='GRCh37'))
    
    # Now you can access the contig and position attributes
    ht_v2d = ht_v2d.annotate(
        study_id = ht_v2d.study_id,
        Chromosome = ht_v2d.locus.contig,
        Start = ht_v2d.locus.position,
        End = ht_v2d.locus.position
    )
    
    df_v2d = _to_pandas(ht_v2d)
    #print(df_v2d.head())

    # kcps2 lead variants
    df = pd.read_table(f"/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/KCPS2_loci/GenomicRiskLoci_{trait}.txt", delimiter="\t")
    #print(df.head())
    
    # pyrange
    gr = pr.PyRanges(df)
    #print(gr.head())

    gr_v2d = pr.PyRanges(df_v2d)
    #print(gr_v2d.head())

    df_annotated = gr.join(gr_v2d, how="left", suffix="_otg").as_df()
    df_annotated = df_annotated.drop(columns=["Chromosome", "Start", "End", "Start_otg", "End_otg"])

    # write out and read back b/c hl.Table.from_pandas is extreamly slow
    tmpfile = new_temp_file()
    with hadoop_open(tmpfile, "wb") as f:
        df_annotated.to_csv(f, sep="\t", na_rep="NA", index=False)

    ht = hl.import_table(tmpfile, impute=True, min_partitions=1000)
    ht = ht.rename({"locus": "locus_otg", "study_id": "study_id_otg"})
    ht = ht.annotate(
        **{
            key: hl.if_else(ht[key] == "-1", hl.missing(hl.tstr), ht[key])
            for key in filter(lambda x: x.endswith("_otg"), list(ht.row))
        }
    )
    ht = ht.annotate(
        lead_locus=hl.parse_locus(ht.lead_locus),
        region=hl.parse_locus_interval(ht.region),
        locus=hl.parse_locus(ht.locus_otg),
        study_id_otg=ht.study_id_otg.split(","),
    )
    ht.show(3)

    for field in ["trait_efo", "trait_efo_category"]:
        x = df_annotated[~df_annotated[field].isna()]
        known_idx = set(x[x[field] == x[f"{field}_otg"]].idx)
        novel_idx = set(x.idx) - set(known_idx)
        na_idx = set(df_annotated.idx) - set(x.idx)
        na_match = na_idx.intersection(
            set(df_annotated[df_annotated[field].isna() & ~df_annotated.locus.isna()].idx)
        )
        k = "efo" if field == "trait_efo" else "efo_category"
        ht = ht.annotate(
            **{
                f"is_known_{k}": hl.case()
                #.when(hl.set(known_idx).contains(ht.idx), True)
                #.when(hl.set(novel_idx).contains(ht.idx), False)
                .when(hl.set(known_idx if len(known_idx) > 0 else hl.empty_set(ht.idx.dtype)).contains(ht.idx), True)
                .when(hl.set(novel_idx if len(novel_idx) > 0 else hl.empty_set(ht.idx.dtype)).contains(ht.idx), False)
                .default(hl.missing(hl.tbool))
            }
        )
        print(field)
        # category known
        print(len(known_idx))
        # category novel
        print(len(novel_idx))
        # category NA
        print(len(na_idx))
        # category NA match
        print(len(na_match))
        
    #N_novel = len(novel_idx)
    #N_novel_cat = len(novel_idx)
    #res_count = pd.DataFrame({'trait': [trait], 'Number of loci': [N_total], 'Number of novel loci': [N_unrep]})
    
    ht.export(f"/n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/KCPS2_otg/known_kcps2_loci_{study}_{trait}_pmid.tsv")


In [15]:
## 09/24/2024
trait_list = ["HEIGHT","TSH","TSH","TSH"]
study_list = ["Yengo","Zhou","William","Sterenborg"]
res = pd.DataFrame()
for i in range(len(trait_list)):
    trait = trait_list[i]; study = study_list[i]
    print(trait); print(study)
    annotate_known_loci_pmid(trait,study)   

HEIGHT
Yengo


/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,,,,,,,,,,
idx,lead_locus,region,trait_efo,trait_efo_category,locus_otg,trait_efo_otg,study_id_otg,trait_efo_term,trait_efo_category_otg,locus
int32,locus<GRCh37>,interval<locus<GRCh37>>,str,str,str,str,array<str>,str,str,locus<GRCh37>
2,1:3417131,[1:3416758-1:3455242),"""EFO_0004339""","""Body measurement""","""1:3454592""","""EFO_0004339""","[""GCST90245848""]","""body height""","""Body measurement""",1:3454592
4,1:17306596,[1:17263557-1:17429774),"""EFO_0004339""","""Body measurement""","""1:17281003""","""EFO_0004339""","[""GCST90245844""]","""body height""","""Body measurement""",1:17281003
4,1:17306596,[1:17263557-1:17429774),"""EFO_0004339""","""Body measurement""","""1:17292404""","""EFO_0004339""","[""GCST90245848""]","""body height""","""Body measurement""",1:17292404


trait_efo
406
22
0
0
trait_efo_category
406
22
0
0


TSH
Zhou


...

trait_efo
57
43
0
0
trait_efo_category
57
43
0
0


TSH
Sterenborg


/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,,,,,,,,,,
idx,lead_locus,region,trait_efo,trait_efo_category,locus_otg,trait_efo_otg,study_id_otg,trait_efo_term,trait_efo_category_otg,locus
int32,locus<GRCh37>,interval<locus<GRCh37>>,str,str,str,str,array<str>,str,str,locus<GRCh37>
1,1:19841174,[1:19407444-1:20020993),"""EFO_0004748""","""Other measurement""","""1:19637058""","""EFO_0004748""","[""EFO_0004748""]","""thyroid stimulating hormone measurement""","""Other measurement""",1:19637058
1,1:19841174,[1:19407444-1:20020993),"""EFO_0004748""","""Other measurement""","""1:19668630""","""EFO_0004748""","[""EFO_0004748""]","""thyroid stimulating hormone measurement""","""Other measurement""",1:19668630
1,1:19841174,[1:19407444-1:20020993),"""EFO_0004748""","""Other measurement""","""1:19771925""","""EFO_0004748""","[""EFO_0004748""]","""thyroid stimulating hormone measurement""","""Other measurement""",1:19771925


trait_efo
60
40
0
0
trait_efo_category
60
40
0
0


In [4]:
trait_list = ["LDL", "TG", "HDL", "CHO", "FBS", "INSULIN","GOT", "GPT", "GGT", "BIL", "ALB","TSH", "CEA","CREAT","ADIPO","URIC", "ALP","HB", "HCT", "MCH", "MCHC", "MCV","RBC", "RDW", "WBC","PLT", "EOS", "SBP", "DBP", "WT",
              "HEIGHT", "BMI", "WAIST", "SMOKA_MOD", "ALCO_AMOUNT","COFFA"]
res = pd.DataFrame()
for i in range(len(trait_list)):
    trait = trait_list[i]
    print(trait)
    annotate_known_loci_EAS(trait)    

LDL


Initializing Hail with default parameters...


24/09/22 15:21:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.4
SparkUI available at http://holy7c04110.rc.fas.harvard.edu:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.130-bea04d9c79b5
LOGGING: writing to /n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/hail-20240922-1520-0.2.130-bea04d9c79b5.log
2024-09-22 15:21:27.032 Hail: INFO: Reading table to impute column types
2024-09-22 15:21:28.374 Hail: INFO: Finished type imputation
  Loading field 'locus' as type str (imputed)
  Loading field 'trait_efo' as type str (imputed)
  Loading field 'study_id' as type str (imputed)
  Loading field 'trait_efo_term' as type str (imputed)
  Loading field 'trait_efo_category' as type str (imputed)
/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is de

,,,,,,,,,,
idx,lead_locus,region,trait_efo,trait_efo_category,locus_otg,trait_efo_otg,study_id_otg,trait_efo_term,trait_efo_category_otg,locus
int32,locus<GRCh37>,interval<locus<GRCh37>>,str,str,str,str,array<str>,str,str,locus<GRCh37>
1,1:16505320,[1:16493225-1:16733077),"""EFO_0004611""","""Lipid or lipoprotein measurement""","""1:16505320""","""EFO_0004735""","[""GCST90255488""]","""serum alanine aminotransferase measurement""","""serum alanine aminotransferase measurement""",1:16505320
1,1:16505320,[1:16493225-1:16733077),"""EFO_0004611""","""Lipid or lipoprotein measurement""","""1:16505320""","""EFO_0004535""","[""GCST90255484""]","""serum albumin measurement""","""serum albumin measurement""",1:16505320
1,1:16505320,[1:16493225-1:16733077),"""EFO_0004611""","""Lipid or lipoprotein measurement""","""1:16505320""","""EFO_0004532""","[""GCST90255490""]","""serum gamma-glutamyl transferase measurement""","""serum gamma-glutamyl transferase measurement""",1:16505320


...

COFFA


2024-09-22 15:29:44.563 Hail: INFO: Reading table to impute column types
2024-09-22 15:29:44.906 Hail: INFO: Finished type imputation
  Loading field 'locus' as type str (imputed)
  Loading field 'trait_efo' as type str (imputed)
  Loading field 'study_id' as type str (imputed)
  Loading field 'trait_efo_term' as type str (imputed)
  Loading field 'trait_efo_category' as type str (imputed)
/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/n/home03/yjee/.conda/envs/hail_envp/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=T

,,,,,,,,,,
idx,lead_locus,region,trait_efo,trait_efo_category,locus_otg,trait_efo_otg,study_id_otg,trait_efo_term,trait_efo_category_otg,locus
int32,locus<GRCh37>,interval<locus<GRCh37>>,str,str,str,str,array<str>,str,str,locus<GRCh37>
1,4:89020503,[4:88991386-4:89020503),"""EFO_0006781""","""Other measurement""","""4:89001893""","""EFO_0004741""","[""GCST90278623""]","""blood urea nitrogen measurement""","""blood urea nitrogen measurement""",4:89001893
2,7:17284577,[7:17280513-7:17464965),"""EFO_0006781""","""Other measurement""","""7:17284577""","""EFO_0004570""","[""GCST90278643""]","""bilirubin measurement""","""bilirubin measurement""",7:17284577
3,12:112241766,[12:109838197-12:113713062),"""EFO_0006781""","""Other measurement""","""12:109880909""","""EFO_0007878""","[""GCST90255464""]","""alcohol consumption measurement""","""alcohol consumption measurement""",12:109880909


trait_efo
0
3
0
0
trait_efo_category
0
3
0
0


2024-09-22 15:29:57.525 Hail: INFO: merging 1001 files totalling 109.8K... 1000]
2024-09-22 15:29:57.619 Hail: INFO: while writing:
    /n/holylfs05/LABS/kraft_lab/Lab/KCPS2/OTG/KCPS2_otg/known_kcps2_loci_COFFA_EAS.tsv
  merge time: 93.790ms
